## Things to add
 - multi shot prompt 
 - saving responses 
 - chat memory 

# Model Testing 

**Here are my PC specs :)** 
 - CPU: AMD 5900x (12 core, 24 thread)
 - GPU: RTX 3080ti (12gb VRAM)
 - MEMORY: 32gb 3600mhz
 - STORAGE: 2 x 2TB NVME
---

In [4]:
# import the stuff we need
import langchain
import ollama

## model selections 
 - Qwen 2.5 coder (0.5b) 
 - Qwen 2.5 coder (32b) 
 - Wizardcoder (33b)
 - Starcoder (15b)
---


In [5]:
# pick what model to use 
model1 = 'qwen2.5-coder:0.5b'
model2 = 'qwen2.5-coder:32b'
model3 = 'wizardcoder:33b'
model4 = 'starcoder2:15b'

## Prompt templates
 - one shot 
 - multi shot
 - chain of thought 
---

In [6]:
# import the prompt block
from langchain.prompts import PromptTemplate

In [43]:
oneShot = PromptTemplate(
    input_variables=["method_code"],
    template='''\
You are a java expert. Write a professional Javadoc comment for the following Java method. Be concise but informative.

Example:
Method:
public String reverseProcessed(String text) {{
    if (text == null) return null;
    String cleaned = text.trim().toLowerCase();
    return new StringBuilder(cleaned).reverse().toString();
}}

Expected Output:
/**
 * Reverses the characters in the given string after trimming whitespace and converting to lowercase.
 *
 * @param text the input string to process
 * @return the reversed, lowercase, trimmed string
 */

make sure to only include the java doc comment, and not the method itself.
now write a expertly written javadoc comment for this method:
{method_code}
'''
)


In [33]:
chainOfThought = PromptTemplate(
    template='''
Write to me a professional javadoc for this Java funcion. Here are the steps you should take:
Is this method for a data structure? If so, what data structure?
What are the parameters?
What does the method do? Look at the name of the method
How does the method do what it does? Are there edge cases?
What exceptions does the method throw?
What is the state of the object before the method is called?
What is the state of the object after the method is called?
What does the method return?
Is this method overriding another method?
Once you have finished the analysis, generate a professional javadoc with a clear description of what the method does, parameters taken in, return values, pre and post conditions, exceptions, and any other notes that should be in the javadoc. 
'''
)

## Sample code 
 - removeFromBST (Chain of Thought)
 - cloneBallSeq (One Shot)
---

In [34]:
removeFromBST = oneShot.format(method_code=
    """private Node doRemove(Node r, String s, Node before, boolean removeOnce, Integer val) {
		if(r == null) {
			lastRemoved = null;
			return null;
		}
		int c = s.compareTo(r.string);
		if(c < 0) r.left = doRemove(r.left, s, before, removeOnce, val);
		else if(c > 0) r.right = doRemove(r.right, s, r, removeOnce, val);
		else {
			if(val != null && val != r.count) return r;
			lastRemoved = r.count;
			if(removeOnce && r.count > 1) {
				r.count--;
				return r;
			}
			if(before.next != r) {
				before = r.left;
				while(before.right != null) before = before.right;
			}
			version++;
			numEntries--;
			if(r.left == null) {
				before.next = r.next;
				return r.right;
			}
			if(r.right == null) {
				before.next = r.next;
				return r.left;
			}
			
			Node successor = r.right;
			Node prev = r;
			while(successor.left != null) {
				prev = successor;
				successor = successor.left;
			}
			r.string = successor.string;
			r.count = successor.count;
			
			if(prev.left == successor) prev.left = successor.right;
			else prev.right = successor.right;
			
			r.next = successor.next;


		}
		return r;
"""
)

In [35]:
cloneBallSeq = chainOfThought.format(method_code='''
public BallSeq clone( )
    {{
        assert wellFormed() : "invariant wrong at start of clone()";

        BallSeq result;

        try
        {{
            result = (BallSeq) super.clone( );
        }}
        catch (CloneNotSupportedException e)
        {{
            // This exception should not occur. But if it does, it would probably
            // indicate a programming error that made super.clone unavailable.
            // The most common error would be forgetting the "Implements Cloneable"
            // clause at the start of this class.
            throw new RuntimeException
            ("This class does not implement Cloneable");
        }}
        // This is inspired by the textbook for this class, page 203
        if(result.head == null) return result;

        Node source = result.head;
        Node copyHead = new Node(source.data, null);
        Node copyTail = copyHead;

        while(source.next != null) {{
            if(source == cursor) result.cursor = copyTail;
            if(source == precursor) result.precursor = copyTail;
            if(source == tail) result.tail = source;
            source = source.next;
            copyTail.next = new Node(source.data, null);
            copyTail = copyTail.next;
        }}
        result.head = copyHead;
        if(source == cursor) result.cursor = copyTail;
        if(source == precursor) result.precursor = copyTail;
        if(source == tail) result.tail = copyTail;

        // TODO: Implemented by student.
        // Now do the hard work of cloning the list.
        // See pp 200-204, 235 (3rd ed. pp. 193-197, 228)
        // Setting precursor, cursor and tail correctly is tricky.


        assert wellFormed() : "invariant wrong at end of clone() for this";
        assert result.wellFormed() : "invariant wrong for result of clone() for copy";
        return result;
    }}
''')


In [36]:
# add some content 
shortMethod = oneShot.format(
    method_code = """
    private void connect(HexPiece p) {
HexCoordinate h = p.getLocation();
for (HexDirection d : HexDirection.values()) {
HexCoordinate h2 = d.move(h);
HexPiece p2 = findPiece(h2);
if (p2 != null) {
p.neighbors[d.ordinal()] = p2;
p2.neighbors[d.reverse().ordinal()] = p;
}
}
}
                         """)

# Evaluating outputs

### Qwen2.5 coder (0.5b)

In [45]:
# Run model
from ollama import chat

response = chat(model=model1, messages=[{'role': 'user', 'content': shortMethod}])
print(response['message']['content'])


```java
/**
 * Connects the pieces of a hexagonal grid to each other based on their directions.
 *
 * @param p A HexPiece object representing a piece in the hexagonal grid.
 */
private void connect(HexPiece p) {
    HexCoordinate h = p.getLocation();
    for (HexDirection d : HexDirection.values()) {
        HexCoordinate h2 = d.move(h);
        HexPiece p2 = findPiece(h2);
        if (p2 != null) {
            p.neighbors[d.ordinal()] = p2;
            p2.neighbors[d.reverse().ordinal()] = p;
        }
    }
}
```


In [47]:
response = chat(model=model1, messages=[{'role': 'user', 'content': cloneBallSeq}])
print(response['message']['content'])

```java
/**
 * This class represents a data structure to store and manage a collection of items.
 * It provides methods for adding, removing, and retrieving elements from the collection.
 *
 * @param <T> The type of elements stored in the collection.
 */
public class ItemCollection<T> {

    // Constructor to initialize an empty collection
    public ItemCollection() {
        this.items = new ArrayList<>();
    }

    // Method to add an element to the collection
    public void addItem(T item) {
        items.add(item);
    }

    // Method to remove an element from the collection
    public boolean removeItem(T item) {
        return items.remove(item);
    }

    // Method to get the number of elements in the collection
    public int getItemCount() {
        return items.size();
    }

    // Method to get an element at a specific index
    public T getElement(int index) {
        if (index < 0 || index >= items.size()) {
            throw new IndexOutOfBoundsException("Index out 

### Qwen 2.5 coder (32b)

In [40]:
# Run model
response = chat(model=model2, messages=[{'role': 'user', 'content': removeFromBST}])
print(response['message']['content'])


```java
/**
 * Recursively removes a node with the specified string from the binary search tree.
 * If {@code removeOnce} is true, decrements the count of the node if it exists more than once.
 * Otherwise, completely removes the node. Updates the last removed count and adjusts pointers accordingly.
 *
 * @param r the current root node of the subtree
 * @param s the string to be removed from the tree
 * @param before the predecessor node used for pointer adjustments during removal
 * @param removeOnce flag indicating whether to decrement the count or completely remove the node
 * @param val optional value to match the exact count of the node to be removed; if null, any count matches
 * @return the updated root node of the subtree after potential removals
 */
```


In [48]:
response = chat(model=model2, messages=[{'role': 'user', 'content': cloneBallSeq}])
print(response['message']['content'])

Certainly! To provide a comprehensive Javadoc for your Java function, I need to make some assumptions based on typical scenarios. Let's assume the following:

### Method Signature
Let's consider a method named `add` for a class `MyList`, which is an implementation of a simple singly linked list data structure.

```java
public boolean add(E element)
```

### Analysis

1. **Data Structure**: This method belongs to a `MyList` class, which implements a simple singly linked list.
2. **Parameters**:
   - `element`: The element to be added to the list.
3. **Method Purpose**: The method is named `add`, suggesting that its purpose is to append an element to the end of the list.
4. **Method Implementation Details**:
   - It will create a new node with the given element and add it to the end of the list.
   - If the list is empty, this new node becomes both the head and tail of the list.
5. **Edge Cases**:
   - Adding an element to an initially empty list.
6. **Exceptions**: The method does not t

### Wizard Coder (15B)

In [49]:

response = chat(model=model3, messages=[{'role': 'user', 'content': removeFromBST}])
print(response['message']['content'])

Here is the corresponding Javadoc comment for the given method:

```java
/**
 * Processes a binary tree node by removing nodes based on string comparison and other conditions.
 * 
 * @param r current node to process
 * @param s string to compare with each node's string value
 * @param before previous node in traversal order
 * @param removeOnce whether to remove one instance of the node or all instances
 * @param val if not null, only removes nodes with this count
 * @return the processed node
 */
private Node doRemove(Node r, String s, Node before, boolean removeOnce, Integer val) { 
    // Method body here...
}
```
This comment describes what each of the parameters does and why they are needed. It also explains that the method processes a binary tree node by removing nodes based on string comparison and other conditions, which is evident from its function within the code. The return value of the method is explained as well. 


In [50]:
response = chat(model=model3, messages=[{'role': 'user', 'content': cloneBallSeq}])
print(response['message']['content'])

This problem is too general to provide an exact solution. However, based on your request, I'll make up a scenario and generate a JavaDoc. Let's assume you have a class named `DataStructure` with a method called `addElement` that adds an element into the data structure. This is just for example purposes:

```java
public class DataStructure {
    /**
     * Adds an element to the data structure.
     * 
     * @param value The value of the element to add.
     * @return true if the addition was successful, false otherwise.
     * 
     * @throws IllegalArgumentException if the provided value is not valid for this data structure.
     * 
     * @precondition The DataStructure object must be initialized and the given parameter must not be null.
     * @postcondition If the addition was successful, the size of the data structure will increase by one.
     */
    public boolean addElement(Object value) {
        // Code for adding an element to the data structure...
        return false;
   

### Starcoder2 (15b)

In [51]:
response = chat(model=model4, messages=[{'role': 'user', 'content': removeFromBST}])
print(response['message']['content'])

In [15]:
response = chat(model=model4, messages=[{'role': 'user', 'content': cloneBallSeq}])
print(response['message']['content'])